# LOGISTIC REGRESSION

## Titanic dataset

dataset link : 
https://iiitaphyd-my.sharepoint.com/:f:/g/personal/apurva_jadhav_students_iiit_ac_in/Eictt5_qmoxNqezgQQiMWeIBph4sxlfA6jWAJNPnV2SF9Q?e=mQmYN0 

(titanic.csv)


In [10]:
# Importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from scipy import stats

from sklearn.model_selection import train_test_split

def min_max_normalize(X):
    min_val = np.min(X, axis=0)
    max_val = np.max(X, axis=0)

    d = max_val - min_val

    X = X - min_val
    X = X / d

    return X

def separte_features_and_outputs(data, drop_list, label_column, normalize_type=0):
    X = data.drop(drop_list, axis=1)
    Y = data[label_column]
       
    return X, Y

datra_frame = pd.read_csv("titanic.csv")

datra_frame['Embarked'].fillna(datra_frame['Embarked'].mode()[0], inplace=True)
datra_frame['Age'].fillna((datra_frame['Age'].mean()), inplace=True)

X, y = separte_features_and_outputs(datra_frame, ['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], 'Survived')

X['Embarked_S'] = X['Embarked']
X['Embarked_S'] = np.where(X['Embarked_S']=='S', 1, 0)
X['Embarked_C'] = X['Embarked']
X['Embarked_C'] = np.where(X['Embarked_C']=='C', 1, 0)
X['Embarked_Q'] = X['Embarked']
X['Embarked_Q'] = np.where(X['Embarked_Q']=='Q', 1, 0)

X = X.drop(['Embarked'], axis=1)
X['Sex'] = np.where(X['Sex']=='male', 1, 0)

X = np.array(X)
X = min_max_normalize(X)

y = np.array(y)

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.30)

In [12]:
class one_vs_one:
    def __init__(self, total_features, total_classes, bias=True):
        
        self.total_features = total_features
        self.bias = bias
        if self.bias==True:
            self.total_features += 1

        self.total_classes = total_classes

        count = int((self.total_classes * (self.total_classes-1))/2)
        self.weights = np.ones(self.total_features*count).reshape((count,self.total_features,1))

    def save_weights(self):
        file_name = "weights_ovo.npy"
        with open(file_name, 'wb') as f:
            np.save(f, self.weights)
    
    def load_weights(self):
        file_name = "weights_ovo.npy"
        with open(file_name, 'rb') as f:
            self.weights = np.load(f)

    def train(self, X_train, Y_train, Transfer_learning=False, epoch=5000, alpha=0.01):

        self.epoch = epoch
        self.alpha = alpha

        if Transfer_learning == True:
            self.load_weights()   

        # print("\nTrain :: ")

        X_train = np.c_[X_train, np.ones(X_train.shape[0])]
        total_count = X_test.shape[0]

        weight_idx = 0

        for i in tqdm(range(self.total_classes)):
            for j in range(i+1, self.total_classes):
                # print("\n", i, "vs" , j ,"::")

                new_X_train = [X_train[idx] for idx in range(total_count) if (Y_train[idx] == i or Y_train[idx] == j)]
                new_X_train = np.array(new_X_train)
                
                m = new_X_train.shape[0]

                new_Y_train = [Y_train[idx] for idx in range(total_count) if (Y_train[idx] == i or Y_train[idx] == j)]
                 
                new_Y_train = [1 if new_Y_train[idx] == i else 0 for idx in range(m)]
                new_Y_train = np.array(new_Y_train)
                new_Y_train = new_Y_train.reshape((-1,1))

                # for _ in tqdm(range(self.epoch)):
                for _ in range(self.epoch):
                    
                    xw = np.matmul(new_X_train, self.weights[weight_idx])
                    neg_xw = -1 * xw

                    hw_xw = 1 / (1 + np.exp(neg_xw))
                    dj_dw = np.matmul(new_X_train.T, (hw_xw - new_Y_train))/m
                                    
                    self.weights[weight_idx] = self.weights[weight_idx] - (self.alpha*dj_dw)  

                weight_idx += 1

        self.save_weights()        

    def test(self, X_test, Y_test):
        # print("\nTest :: ")

        X_test = np.c_[X_test, np.ones(X_test.shape[0])]
        m = X_test.shape[0]

        class_pred = []

        weight_idx = 0
        for i in tqdm(range(self.total_classes)):
            for j in range(i+1, self.total_classes):   
                
                xw = np.matmul(X_test, self.weights[weight_idx])
                neg_xw = -1 * xw

                hw_xw = 1 / (1 + np.exp(neg_xw))
                
                onevsone_class_pred = [i if hw_xw[idx]>=0.5 else j for idx in range(m)] 
                class_pred.append(onevsone_class_pred)

                weight_idx += 1

        class_pred = np.array(class_pred)        
        Y_pred = stats.mode(class_pred)[0][0]

        true_count = 0
        for i in range(m):
            if Y_pred[i] == Y_test[i]:
                true_count += 1
        
        accu = true_count/ m
        return accu, confusion_matrix(Y_test, Y_pred), classification_report(Y_test, Y_pred)

In [13]:
one_vs_one_obj = one_vs_one(X_train.shape[1], 2)
one_vs_one_obj.train(X_train,Y_train, Transfer_learning=False, epoch=50000, alpha=0.01)

100%|██████████| 2/2 [00:01<00:00,  1.13it/s]


In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

accu, confusion_matrix, classification_report = one_vs_one_obj.test(X_test, Y_test)

print("\nAccuracy:", accu)
print("\nclassification_report:")
print(classification_report)
print("\nconfusion_matrix:")
print(confusion_matrix)

100%|██████████| 2/2 [00:00<00:00, 2326.29it/s]


Accuracy: 0.7574626865671642

classification_report:
              precision    recall  f1-score   support

           0       0.76      0.83      0.79       151
           1       0.75      0.66      0.70       117

    accuracy                           0.76       268
   macro avg       0.76      0.75      0.75       268
weighted avg       0.76      0.76      0.75       268


confusion_matrix:
[[126  25]
 [ 40  77]]
